In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

In [77]:
import pandas as pd

In [2]:
import requests as rq
import random
import re

In [117]:
# 위에 것이 안되면 아래 것을 실행하면 됨

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome(ChromeDriverManager().install())


In [118]:
Xpaths = {
    'search' : '//*[@id="__next"]/div/div/div/div[2]/div/svg/path',
    'magnify' : '//*[@id="__next"]/div/div/div/div/div[2]/div[1]',
    'search_input' : '//input[@placeholder="기기명, 주소, 지역 등 검색"]'
}
url = 'https://www.superbin.co.kr'
search_area_keywords = ['강남구', '서초구', '관악구']

In [119]:
# 맵 사이트 접속 및 
driver.get(url)
time.sleep(random.uniform(0.5, 2))
driver.find_element(By.XPATH, value='//*[@id="__next"]/header/div/div[2]').click()
time.sleep(random.uniform(0.5, 2))
driver.find_element(By.XPATH, value='//*[@id="modal"]/div/div[2]/ul/li[2]').click()
time.sleep(random.uniform(0.5, 2))

# 창 최대화
driver.maximize_window()

try:
    driver.refresh()
    time.sleep(random.uniform(3, 3.1))

    # 축척 확대
    for i in range(3):
        driver.find_element(By.XPATH, value=Xpaths['magnify']).click()
        time.sleep(random.uniform(0.5, 2))
except:
    driver.refresh()
    time.sleep(random.uniform(3, 3.1))

    # 축척 확대
    for i in range(3):
        driver.find_element(By.XPATH, value=Xpaths['magnify']).click()
        time.sleep(random.uniform(0.5, 2))

# 입력 필드 찾기 (placeholder 속성 사용)
search_box = driver.find_element(By.XPATH, Xpaths['search_input'])

# 입력 필드가 비활성화되어 있다면 활성화하는 코드 필요
# 비활성화된 요소를 활성화하는 방식은 페이지마다 다를 수 있음 (여기서는 간단히 JavaScript 사용 예시)
driver.execute_script("arguments[0].disabled = false;", search_box)

addresses = []
capacities = []

for search_text in search_area_keywords:
    # 입력필드 제거 후 검색어 입력
    search_box.clear()
    search_box.send_keys(search_text)
    search_box.send_keys(Keys.RETURN)

    time.sleep(random.uniform(0.5, 2))

    # 모든 리스트 요소 찾기
    list_items = driver.find_elements(By.CSS_SELECTOR, 'ul.list li')

    # 각 리스트를 클릭 후 상세 정보 표시
    for item in list_items:
        # 리스트 요소 클릭
        item.click()
        time.sleep(random.uniform(0.5, 0.7))
        # BeautifulSoup을 사용하여 HTML 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 필요한 정보 추출
        elements_containing_text = soup.find_all(text=lambda text: "1인 1일" in text)
        try:
            item_limit = elements_containing_text[0]
        except:
            item_limit = "Unspecified"
        capacities.append(item_limit)

        time.sleep(random.uniform(0.5, 2))

    # 파싱
    soup = BeautifulSoup(html, 'html.parser')
    address_list = []
    address_elements = soup.select('ul.list li div.container p.address')

    for address_element in address_elements:
        address_list.append(address_element.text)
    addresses.extend(address_list)

    # 주소를 한 줄씩 input.txt 파일에 저장
    with open('./../Data/input.txt', 'w') as file:
        for address in address_list:
            file.write(address + '\n')
driver.quit()

C:\Users\james\AppData\Local\Temp\ipykernel_29172\2907234911.py:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elements_containing_text = soup.find_all(text=lambda text: "1인 1일" in text)


In [120]:
# 주소와 용량으로 DataFrame 생성
df = pd.DataFrame({"Address": addresses, "Capacity": capacities})
df

,Address,Capacity
0,서울특별시 강남구 삼성로 151,1인 1일 30개 (1회 30개)
1,서울특별시 강남구 압구정로28길 28,1인 1일 30개 (1회 30개)
2,서울특별시 강남구 논현로105길 53,1인 1일 30개 (1회 30개)
3,서울특별시 서초구 서초대로38길 12,1인 1일 30개 (1회 30개)
4,서울특별시 서초구 서초대로38길 12,1인 1일 30개 (1회 30개)
5,서울특별시 서초구 반포동 20-48,1인 1일 50개 (1회 50개)
6,서울특별시 관악구 봉천동 912-26,1인 1일 30개 (1회 30개)
7,서울특별시 관악구 낙성대로4가길 5 낙성대동주민센터,1인 1일 30개 (1회 30개)
8,서울특별시 관악구 난곡로30길 40,1인 1일 30개 (1회 30개)
9,서울특별시 관악구 신림동 646-131,1인 1일 30개 (1회 30개)
